In [4]:
!pip install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/lts/1.8/cpu
     |████████████████████████████████| 169.1 MB 3.0 kB/s eta 0:00:01    |████▍                           | 23.3 MB 1.3 MB/s eta 0:01:57     |██████▏                         | 32.7 MB 1.4 MB/s eta 0:01:37     |████████████▏                   | 64.2 MB 1.2 MB/s eta 0:01:28     |████████████▎                   | 64.7 MB 1.5 MB/s eta 0:01:10     |█████████████▍                  | 70.6 MB 1.4 MB/s eta 0:01:09     |██████████████████▍             | 97.4 MB 1.4 MB/s eta 0:00:52     |██████████████████▋             | 98.5 MB 771 kB/s eta 0:01:32     |██████████████████████▊         | 120.0 MB 563 kB/s eta 0:01:28     |███████████████████████▊        | 125.6 MB 473 kB/s eta 0:01:32     |███████████████████████▉        | 126.0 MB 473 kB/s eta 0:01:32     |█████████████████████████▋      | 135.1 MB 1.3 MB/s eta 0:00:27
     |████████████████████████████████| 13.3 MB 755 kB/s eta 0:00:01    |████████████    

In [5]:
!pip install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 4.7 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 768 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 616 kB/s ta 0:00:01
     |████████████████████████████████| 6.6 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 409 kB/s eta 0:00:01


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re 

In [7]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [8]:
tokens = tokenizer.encode('It was good but could have been better', return_tensors='pt')

In [9]:
result = model(tokens)

In [10]:
result.logits

tensor([[-1.9994,  0.0689,  2.2148,  1.2780, -1.3454]],
       grad_fn=<AddmmBackward>)

In [11]:
int(torch.argmax(result.logits))+1

3

In [16]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [17]:
import numpy as np
import pandas as pd

In [20]:
df = pd.DataFrame(np.array(reviews), columns = ['review'])

In [21]:
df['review'].iloc[0]

'Great staff and food. \xa0Must try is the pan fried Gnocchi! \xa0The staff were really friendly and the coffee was good as well'

In [23]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [24]:
sentiment_score(df['review'].iloc[1])

3

In [25]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [26]:
df

,review,sentiment
0,Great staff and food. Must try is the pan fri...,5
1,It was ok. The coffee wasn't the best but it w...,3
2,I came to Social brew cafe for brunch while ex...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I went here a little while ago- a beautiful mo...,2
5,We came for brunch twice in our week-long visi...,4
6,Ron & Jo are on the go down under and Wow! We...,5
7,Great coffee and vibe. That's all you need. C...,5
8,Great coffee and vibe. That's all you need. C...,4
9,Good coffee and toasts. Straight up and down -...,5


In [27]:
df['review'].iloc[3]

"Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. Cheers!"